In [7]:
import datetime

# caregiver class
class Caregiver:
    def __init__(self, name, caregiver_type, availability):
        self.name = name
        self.caregiver_type = caregiver_type
        self.availability = availability

    def update_availability(self, day, shift):
        if day not in self.availability:
            self.availability[day] = []
        if shift not in self.availability[day]:
            self.availability[day].append(shift)
        else:
            self.availability[day].remove(shift)

    def generate_html_calendar(self):
        """
        Generates an HTML weekly availability schedule for the caregiver.
        """
        filename = f"{self.name}_availability.html"

        # HTML structure
        html_content = f"""
        <html>
        <head>
            <title>{self.name}'s Availability Schedule</title>
            <style>
                table {{
                    border-collapse: collapse;
                    width: 100%;
                    margin: 20px 0;
                }}
                th, td {{
                    border: 1px solid black;
                    padding: 10px;
                    text-align: center;
                }}
                th {{
                    background-color: #f2f2f2;
                }}
                td {{
                    height: 50px;
                    vertical-align: top;
                }}
            </style>
        </head>
        <body>
            <h1>{self.name}'s Availability Schedule</h1>
            <table>
                <tr>
                    <th>Mon</th>
                    <th>Tue</th>
                    <th>Wed</th>
                    <th>Thu</th>
                    <th>Fri</th>
                    <th>Sat</th>
                    <th>Sun</th>
                </tr>
                <tr>
        """

        # Generating rows for the caregiver's availability
        for day in range(1, 8):  # 7 days of the week
            availability = self.availability.get(day, {})
            morning_status = availability.get("7AM-1PM", "unavailable").capitalize()
            afternoon_status = availability.get("1PM-7PM", "unavailable").capitalize()

            html_content += f"""
                    <td>
                        <b>Morning:</b> {morning_status}<br>
                        <b>Afternoon:</b> {afternoon_status}
                    </td>
            """

        html_content += """
                </tr>
            </table>
        </body>
        </html>
        """

        # Write the HTML file
        with open(filename, "w") as html_file:
            html_file.write(html_content)
        print(f"HTML availability schedule saved as {filename}")


# schedule class
class Schedule:
    def __init__(self, caregivers):
        self.caregivers = caregivers
        self.schedule = {}

    def initialize_schedule(self, days_in_month=31):
        """
        Initializes the schedule for each day of the month with two shifts (AM and PM).
        """
        for day in range(1, days_in_month + 1):
            self.schedule[day] = {"7AM-1PM": None, "1PM-7PM": None}


    def assign_caregivers(self):
        """
        Assigns caregivers to shifts based on their weekly repeating availability.
        Prioritizes 'preferred' caregivers over 'available' ones.
        """
        for day, shifts in self.schedule.items():
            # Determine the day of the week (1=Monday, ..., 7=Sunday)
            day_of_week = (day - 1) % 7 + 1

            for shift in shifts.keys():
                available_caregivers = []

                # Check weekly repeating availability for each caregiver
                for caregiver in self.caregivers:
                    day_availability = caregiver.availability.get(day_of_week, {})
                    status = day_availability.get(shift, "unavailable")

                    # Collect caregivers based on availability status
                    if status == "preferred":
                        available_caregivers.append((caregiver, "preferred"))
                    elif status == "available":
                        available_caregivers.append((caregiver, "available"))

                # Sort caregivers: prioritize 'preferred' over 'available'
                available_caregivers.sort(key=lambda x: x[1], reverse=True)

                # Assign the first available caregiver
                if available_caregivers:
                    self.schedule[day][shift] = available_caregivers[0][0].name
                else:
                    self.schedule[day][shift] = "No caregiver assigned"  # Fallback for unassigned shifts



    def get_available_caregivers(self, day, shift):
        """
        Returns a list of caregivers available for a specific day and shift.
        """
        available = []
        for caregiver in self.caregivers:
            if day in caregiver.availability and shift in caregiver.availability[day]:
                available.append(caregiver.name)
        return available

    def display_schedule(self):
        """
        Displays the schedule in the terminal.
        """
        for day, shifts in self.schedule.items():
            print(f"Schedule for Day {day}:")
            for shift, caregiver in shifts.items():
                caregiver_name = caregiver if caregiver else "No caregiver assigned"
                print(f"  {shift}: {caregiver_name}")


    def generate_html_schedule(self, filename="care_schedule.html", days_in_month=31, first_day_offset=0):
        """
        Generates an HTML file to display the care schedule in a calendar format.
        :param filename: String, the name of the output HTML file.
        :param days_in_month: Integer, number of days in the month.
        :param first_day_offset: Integer, the offset for the first day of the month (0=Monday, 1=Tuesday, ...).
        """
        html_content = """
        <html>
        <head>
            <title>Work Schedule for October 2024</title>
            <style>
                table {
                    border-collapse: collapse;
                    width: 100%;
                    margin: 20px 0;
                }
                th, td {
                    border: 1px solid black;
                    padding: 10px;
                    text-align: center;
                }
                th {
                    background-color: #f2f2f2;
                }
                td {
                    height: 100px;
                    vertical-align: top;
                }
            </style>
        </head>
        <body>
            <h1>Work Schedule for October 2024</h1>
            <table>
                <tr>
                    <th>Mon</th>
                    <th>Tue</th>
                    <th>Wed</th>
                    <th>Thu</th>
                    <th>Fri</th>
                    <th>Sat</th>
                    <th>Sun</th>
                </tr>
        """

        current_day = 1
        for week in range(6):  # up to 6 weeks in a month
            html_content += "<tr>"
            for day in range(7):  # 7 days per week
                if week == 0 and day < first_day_offset or current_day > days_in_month:
                    html_content += "<td></td>"  # empty cell for padding
                else:
                    # get shifts for the current day
                    shifts = self.schedule[current_day]
                    am_shift = f"AM: {shifts['7AM-1PM'] or 'No caregiver assigned'}"
                    pm_shift = f"PM: {shifts['1PM-7PM'] or 'No caregiver assigned'}"

                    # add day and shift details
                    html_content += f"<td><b>{current_day}</b><br>{am_shift}<br>{pm_shift}</td>"
                    current_day += 1
            html_content += "</tr>"

        html_content += """
            </table>
        </body>
        </html>
        """

        with open(filename, "w") as html_file:
            html_file.write(html_content)
        print(f"HTML care schedule saved as {filename}")


class Report:
    def __init__(self, schedule, caregivers):
        self.schedule = schedule
        self.caregivers = caregivers

    # function to calculate hours worked and pay per caregiver
    def calculate_pay(self):
        "Calculates weekly and monthly pay for caregivers."
        # pay_data is dict to hold hours/pay info
        pay_data = {caregiver.name: {"type": caregiver.caregiver_type,
                                     "weekly": [0] * 6,
                                     "monthly": 0} 
                                    for caregiver in self.caregivers}
        
    # calculate hours and pay
        for day, shifts in self.schedule.items():
            week_index = (day + 1) // 7
            for shift, caregiver_name in shifts.items():
                if caregiver_name and caregiver_name != "No caregiver assigned":
                    hours = 6
                    caregiver = next((c for c in self.caregivers if c.name == caregiver_name), None)
                    if caregiver and caregiver.caregiver_type == "paid":
                        pay = hours * 20
                        pay_data[caregiver.name]["weekly"][week_index - 1] += pay
                        pay_data[caregiver.name]["monthly"] += pay

        return pay_data

    # function to create text file for pay report
    def generate_pay_report(self, file_name = "pay_report.txt"):
        "Generates txt pay report"
        pay_data = self.calculate_pay()

        # write new file
        with open(file_name, 'w') as file:
            file.write("Pay Report\n")
            file.write("=" * 40 + '\n')

            total_weekly = [0] * 6
            total_monthly = 0

            # loop through the caregivers from the pay_data dict
            for caregiver, data in pay_data.items():
                file.write(f"Caregiver: {caregiver} ({data['type']})\n")
                # iterate through weekly pay
                for week, weekly_pay in enumerate(data["weekly"], start=1):
                    file.write(f"  Week {week}: ${weekly_pay:.2f}\n")
                    total_weekly[week - 1] += weekly_pay
                file.write(f"  Monthly Total: ${data['monthly']:.2f}\n\n")
                total_monthly += data["monthly"]

            # print totals
            # by week
            file.write("=" * 40 + '\n')
            file.write("Totals:\n")
            for week, weekly_total in enumerate(total_weekly, start=1):
                file.write(f"  Week {week}: ${weekly_total:.2f}\n")
            file.write(f"  Monthly Total: ${total_monthly:.2f}\n\n")

            # by person
            file.write("Monthly total by caregiver:\n")
            for caregiver, data in pay_data.items():
                file.write(f"{caregiver}: ${data['monthly']:.2f}\n")
            # file.write("\n")

# main
if __name__ == "__main__":
    # create caregiver instances
    # adding weekly caregiver availability here so the care schedule can be formed based on this
    caregiver1 = Caregiver("Abby", "paid", {
        1: {"7AM-1PM": "preferred", "1PM-7PM": "available"},
        2: {"7AM-1PM": "available", "1PM-7PM": "unavailable"},
        3: {"7AM-1PM": "preferred", "1PM-7PM": "available"},
        4: {"7AM-1PM": "available", "1PM-7PM": "available"},
        5: {"7AM-1PM": "preferred", "1PM-7PM": "unavailable"},
        6: {"7AM-1PM": "available", "1PM-7PM": "available"},
        7: {"7AM-1PM": "preferred", "1PM-7PM": "available"}
    })
    caregiver2 = Caregiver("Greg", "paid", {
        1: {"7AM-1PM": "available", "1PM-7PM": "preferred"},
        2: {"7AM-1PM": "preferred", "1PM-7PM": "available"},
        3: {"7AM-1PM": "available", "1PM-7PM": "available"},
        4: {"7AM-1PM": "available", "1PM-7PM": "preferred"},
        5: {"7AM-1PM": "available", "1PM-7PM": "available"},
        6: {"7AM-1PM": "preferred", "1PM-7PM": "available"},
        7: {"7AM-1PM": "available", "1PM-7PM": "preferred"}
    })
    caregiver3 = Caregiver("Luke", "family", {
        1: {"7AM-1PM": "available", "1PM-7PM": "available"},
        2: {"7AM-1PM": "preferred", "1PM-7PM": "available"},
        3: {"7AM-1PM": "available", "1PM-7PM": "available"},
        4: {"7AM-1PM": "preferred", "1PM-7PM": "preferred"},
        5: {"7AM-1PM": "available", "1PM-7PM": "available"},
        6: {"7AM-1PM": "available", "1PM-7PM": "available"},
        7: {"7AM-1PM": "preferred", "1PM-7PM": "preferred"}
    })
    caregiver4 = Caregiver("Emma", "family", {
        1: {"7AM-1PM": "preferred", "1PM-7PM": "available"},
        2: {"7AM-1PM": "available", "1PM-7PM": "available"},
        3: {"7AM-1PM": "preferred", "1PM-7PM": "available"},
        4: {"7AM-1PM": "available", "1PM-7PM": "available"},
        5: {"7AM-1PM": "available", "1PM-7PM": "preferred"},
        6: {"7AM-1PM": "available", "1PM-7PM": "preferred"},
        7: {"7AM-1PM": "preferred", "1PM-7PM": "available"}
    })
    caregiver5 = Caregiver("Sophia", "paid", {
        1: {"7AM-1PM": "available", "1PM-7PM": "preferred"},
        2: {"7AM-1PM": "preferred", "1PM-7PM": "preferred"},
        3: {"7AM-1PM": "available", "1PM-7PM": "available"},
        4: {"7AM-1PM": "available", "1PM-7PM": "available"},
        5: {"7AM-1PM": "preferred", "1PM-7PM": "available"},
        6: {"7AM-1PM": "available", "1PM-7PM": "preferred"},
        7: {"7AM-1PM": "preferred", "1PM-7PM": "preferred"}
    })
    caregiver6 = Caregiver("Ethan", "paid", {
        1: {"7AM-1PM": "preferred", "1PM-7PM": "available"},
        2: {"7AM-1PM": "available", "1PM-7PM": "available"},
        3: {"7AM-1PM": "preferred", "1PM-7PM": "available"},
        4: {"7AM-1PM": "preferred", "1PM-7PM": "preferred"},
        5: {"7AM-1PM": "available", "1PM-7PM": "preferred"},
        6: {"7AM-1PM": "preferred", "1PM-7PM": "available"},
        7: {"7AM-1PM": "preferred", "1PM-7PM": "available"}
    })
    caregiver7 = Caregiver("Olivia", "family", {
        1: {"7AM-1PM": "available", "1PM-7PM": "preferred"},
        2: {"7AM-1PM": "preferred", "1PM-7PM": "available"},
        3: {"7AM-1PM": "available", "1PM-7PM": "available"},
        4: {"7AM-1PM": "preferred", "1PM-7PM": "preferred"},
        5: {"7AM-1PM": "available", "1PM-7PM": "preferred"},
        6: {"7AM-1PM": "available", "1PM-7PM": "preferred"},
        7: {"7AM-1PM": "preferred", "1PM-7PM": "available"}
    })
    caregiver8 = Caregiver("Noah", "family", {
        1: {"7AM-1PM": "available", "1PM-7PM": "available"},
        2: {"7AM-1PM": "preferred", "1PM-7PM": "preferred"},
        3: {"7AM-1PM": "available", "1PM-7PM": "available"},
        4: {"7AM-1PM": "preferred", "1PM-7PM": "available"},
        5: {"7AM-1PM": "available", "1PM-7PM": "preferred"},
        6: {"7AM-1PM": "available", "1PM-7PM": "available"},
        7: {"7AM-1PM": "preferred", "1PM-7PM": "preferred"}
    })



    # add caregivers to the list
    caregivers = [caregiver1, caregiver2, caregiver3, caregiver4, caregiver5, caregiver6, caregiver7, caregiver8]


    # generate individual HTML calendars for each caregiver
    for caregiver in caregivers:
        caregiver.generate_html_calendar()

    # initialize schedule
    schedule = Schedule(caregivers)
    schedule.initialize_schedule(days_in_month=31)  # adjust based on which month

    # assign caregivers based on their availability
    schedule.assign_caregivers()

    # generates HTML file for schedule
    schedule.generate_html_schedule(filename="care_schedule.html", days_in_month=31, first_day_offset=2) # offset 2 for October 2024

    # generate pay report file
    report = Report(schedule.schedule, caregivers)
    report.generate_pay_report(file_name="pay_report.txt")



HTML availability schedule saved as Abby_availability.html
HTML availability schedule saved as Greg_availability.html
HTML availability schedule saved as Luke_availability.html
HTML availability schedule saved as Emma_availability.html
HTML availability schedule saved as Sophia_availability.html
HTML availability schedule saved as Ethan_availability.html
HTML availability schedule saved as Olivia_availability.html
HTML availability schedule saved as Noah_availability.html
HTML care schedule saved as care_schedule.html
